# 104人力銀行爬蟲程式規格文件

## 1. 程式目的
- 抓取104人力銀行網站上的職缺資訊
- 針對特定關鍵字和職務類別進行搜尋
- 將資料整理成結構化格式儲存

## 2. 使用的套件
- `time`: 時間處理和延遲
- `random`: 隨機延遲時間產生
- `json`: JSON 資料處理
- `requests`: HTTP 請求處理
- `bs4 (BeautifulSoup)`: HTML 解析
- `tqdm`: 進度條顯示
- `pandas`: 資料處理與分析
- `collections.deque`: 固定長度佇列
- `concurrent.futures.ThreadPoolExecutor`: 多執行緒處理

## 3. 主要功能模組

### 3.1 職業類別目錄獲取
- API 端點: `https://static.104.com.tw/category-tool/json/JobCat.json`
- 功能: 獲取完整的職業分類資訊
- 輸出: DataFrame 格式的職業分類資料

### 3.2 搜尋 URL 生成 (`catch_104_url`)
- 功能: 根據搜尋參數生成104網站的搜尋 URL
- 參數:
  - `KEYWORDS`: 職缺關鍵字
  - `CATEGORY`: 職缺類別代碼
  - `ORDER`: 排序方式（可選）
    - 15: 符合度高
    - 16: 最近更新
- 返回: 完整的搜尋 URL

### 3.3 職缺列表獲取 (`fetch_jobs_url`)
- 功能: 抓取符合條件的職缺列表
- API 端點: `https://www.104.com.tw/jobs/search/api/jobs`
- 參數:
  - `_CODE`: 職缺類別代碼
  - `KEYWORD`: 搜尋關鍵字
- 搜尋限制:
  - 最大頁數: 10頁
  - 每頁筆數: 30筆
- 特殊處理:
  - 使用 `tqdm` 顯示進度
  - 隨機延遲 0.5-1.5 秒
  - 去除重複職缺
  - 連續4次無新資料時提前結束

### 3.4 職缺詳細資訊獲取 (`fetch_job_data`)
- 功能: 獲取單一職缺的詳細資訊
- API 格式: `https://www.104.com.tw/job/ajax/content/{職缺ID}`
- 輸出: 包含職缺完整資訊的 DataFrame

## 4. 資料處理流程

1. 初始化設定
   - 設定搜尋時間戳記
   - 設定職缺類別代碼
   - 設定搜尋關鍵字
   - 設定輸出檔案名稱格式

2. 資料獲取流程
   - 取得職缺列表
   - 轉換為 API 格式的 URL
   - 逐一獲取詳細資訊
   - 合併到總表

3. 資料輸出
   - 以 DataFrame 格式儲存
   - 可匯出為 CSV 或 Excel 格式

## 5. 安全性考量

### 5.1 請求標頭設定
```python
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
    'Referer': 'https://www.104.com.tw/jobs/search'
}
```

### 5.2 爬蟲限制
- 使用隨機延遲避免過度請求
- 設定請求超時限制 (20秒)
- 偵測重複資料避免無效請求

## 6. 錯誤處理
- 使用 `requests.Session()` 管理連線
- 設定請求超時機制
- 檢查 API 回應狀態
- 資料重複性檢查
- 連續無新資料時提前結束機制

## 7. 輸出格式
檔案命名格式：
```
(日期)_104人力銀行_關鍵字_職類代碼
```
範例：
```
(2025-05-22)_104人力銀行_雲端工程師_2007000000.csv
```